In [1]:
import torch
from torch import nn
from models.utils import param, reverse_packed_sequence, Namespace, packCharsWithMask, pad_sequence, pad_packed_sequence, pack_padded_sequence
from torch.nn.utils.rnn import PackedSequence
from models.networks import GlobalContextualDeepTransition
import pytorch_lightning as pl
import torch.nn.functional as F

In [2]:
wordEmbedding = 20 # 300
charEmbedding = 20 # 128
contextOutputUnits = outputUnits = 70
numWords = 10
numChars = 10
numTags = 17
contextTransitionNumber = transitionNumber = 3
maxLen = 5
encoderUnits = decoderUnits = 50

In [3]:
#max chars in word is 6
s1 = [
    [1, 2, 3],
    [4, 3],
    [5, 2, 2, 3, 7, 6],
]


s2 = [
    [4, 3, 1, 6, 2],
    [1, 2, 3],
]


# the packed sequence removes word axis padding, custom mask handles char axis padding
x, mask = packCharsWithMask([s1, s2])

w = [
    [2, 3, 1],
    [6, 2]
]

l = [len(s) for s in w]
w = [torch.LongTensor(s) for s in w]
w = pad_sequence(w)
w = pack_padded_sequence(w, l, enforce_sorted=False)

_, *args = w
m = w.data.size(0)
sparse = torch.randint(numTags, (m, ) )
targets = torch.zeros(m, numTags)
targets[torch.arange(m),sparse] = 1
targets = PackedSequence(targets, *args)

print(w.data.shape, x.data.shape, mask.shape, targets.data.shape)
print("lengths are: ", l)
print(targets.data)

torch.Size([5]) torch.Size([5, 6]) torch.Size([5, 6, 1]) torch.Size([5, 17])
lengths are:  [3, 2]
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [4]:
model = GlobalContextualDeepTransition(numChars, charEmbedding, numWords,
                     wordEmbedding, contextOutputUnits, contextTransitionNumber,
                        encoderUnits, numTags, transitionNumber)

batch = w, x, mask, targets
loss = model.training_step(batch, 0)

print(loss.item())

2.930936098098755
